<h2><a href="https://leetcode.com/problems/combine-two-tables">175. Combine Two Tables</a></h2><h3>Easy</h3><hr><p>Table: <code>Person</code></p>

<pre>
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| personId    | int     |
| lastName    | varchar |
| firstName   | varchar |
+-------------+---------+
personId is the primary key (column with unique values) for this table.
This table contains information about the ID of some persons and their first and last names.
</pre>

<p>&nbsp;</p>

<p>Table: <code>Address</code></p>

<pre>
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| addressId   | int     |
| personId    | int     |
| city        | varchar |
| state       | varchar |
+-------------+---------+
addressId is the primary key (column with unique values) for this table.
Each row of this table contains information about the city and state of one person with ID = PersonId.
</pre>

<p>&nbsp;</p>

<p>Write a solution to report the first name, last name, city, and state of each person in the <code>Person</code> table. If the address of a <code>personId</code> is not present in the <code>Address</code> table, report <code>null</code> instead.</p>

<p>Return the result table in <strong>any order</strong>.</p>

<p>The result format is in the following example.</p>

<p>&nbsp;</p>
<p><strong class="example">Example 1:</strong></p>

<pre>
<strong>Input:</strong> 
Person table:
+----------+----------+-----------+
| personId | lastName | firstName |
+----------+----------+-----------+
| 1        | Wang     | Allen     |
| 2        | Alice    | Bob       |
+----------+----------+-----------+
Address table:
+-----------+----------+---------------+------------+
| addressId | personId | city          | state      |
+-----------+----------+---------------+------------+
| 1         | 2        | New York City | New York   |
| 2         | 3        | Leetcode      | California |
+-----------+----------+---------------+------------+
<strong>Output:</strong> 
+-----------+----------+---------------+----------+
| firstName | lastName | city          | state    |
+-----------+----------+---------------+----------+
| Allen     | Wang     | Null          | Null     |
| Bob       | Alice    | New York City | New York |
+-----------+----------+---------------+----------+
<strong>Explanation:</strong> 
There is no address in the address table for the personId = 1 so we return null in their city and state.
addressId = 1 contains information about the address of personId = 2.
</pre>

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col

# Initialize Spark Session
spark = SparkSession.builder.master("local").appName("CombineTwoTables").getOrCreate()

# Define Schema for Person Table
person_schema = StructType([
    StructField("personId", IntegerType(), False),
    StructField("lastName", StringType(), False),
    StructField("firstName", StringType(), False)
])

# Define Schema for Address Table
address_schema = StructType([
    StructField("addressId", IntegerType(), False),
    StructField("personId", IntegerType(), False),
    StructField("city", StringType(), True),
    StructField("state", StringType(), True)
])

# Hardcoded Person Data
person_data = [
    (1, "Wang", "Allen"),
    (2, "Alice", "Bob")
]

# Hardcoded Address Data
address_data = [
    (1, 2, "New York City", "New York"),
    (2, 3, "Leetcode", "California")  # This row is ignored since personId = 3 is not in Person table
]

# Create DataFrames
person_df = spark.createDataFrame(person_data, schema=person_schema)
address_df = spark.createDataFrame(address_data, schema=address_schema)

# Display DataFrames
print("Person Table:")
person_df.show()

print("Address Table:")
address_df.show()

25/02/16 23:11:55 WARN Utils: Your hostname, Nahils-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.103 instead (on interface en0)
25/02/16 23:11:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/16 23:11:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Person Table:


+--------+--------+---------+
|personId|lastName|firstName|
+--------+--------+---------+
|       1|    Wang|    Allen|
|       2|   Alice|      Bob|
+--------+--------+---------+

Address Table:
+---------+--------+-------------+----------+
|addressId|personId|         city|     state|
+---------+--------+-------------+----------+
|        1|       2|New York City|  New York|
|        2|       3|     Leetcode|California|
+---------+--------+-------------+----------+



In [2]:
person_df.createOrReplaceTempView('persons')
address_df.createOrReplaceTempView('address')

In [3]:
spark.sql("""
    SELECT 
          p.firstName,
          p.lastName,
          a.city,
          a.state
          FROM persons p
          LEFT JOIN address a ON p.personId = a.personId

""").show()

+---------+--------+-------------+--------+
|firstName|lastName|         city|   state|
+---------+--------+-------------+--------+
|    Allen|    Wang|         NULL|    NULL|
|      Bob|   Alice|New York City|New York|
+---------+--------+-------------+--------+

